<a href="https://colab.research.google.com/github/bhavikar04/Facial-Keypoints-Recogniser/blob/main/mountain_dew_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'keypoint-identification-cv-contingent-selections:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F62891%2F6855609%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240622%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240622T045752Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e1a752d11a7d6905fbc4de9ea98e78dd583604ee7b9040e3c862e636df105944c9df3a38cdc363fbfe27278a0be47f992586fe89dab7d2365a0a737f7482dd42a43c4a88566bd6f9aabf49447afbaf94ed2a015a9f749bfc4aa57764d09a5c79004b8770998ef429641ae5e3e490ac9828ee1bdd5e5bf1711e6509e9f47b5bde516452d2157dbd2fab8084b3ce1846f1be3691ee094e91a4947d6035909f5805148ae5409d552b44eed700b75601785156368fab4538fb308b516abd32968e51cd2a98a3bbddf5e15a1132d73fb8c90ba261edee64bbf00885d63269a1e13dafa1c233f64e95f2ac4cc3b0277ccfec358452494ccfee3d34898bd2f02889250'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import matplotlib.pyplot as plt


In [ ]:
train_data_df = pd.read_csv('/kaggle/input/keypoint-identification-cv-contingent-selections/training.csv')
test_data_df = pd.read_csv('/kaggle/input/keypoint-identification-cv-contingent-selections/test.csv')

In [ ]:
def process_data(data_df,train):
    if train:
        y=np.array(data_df.iloc[:,:30])

    img_dt = []
    for i in range(len(data_df)):
        img_dt.append(data_df['Image'][i].split(' '))

    X=np.array(img_dt, dtype='float')
    return X,y if train else " "
X_train,y_train=process_data(train_data_df,True)
X_test_submit,_=process_data(test_data_df,False)

In [ ]:
class TrainDataset():

    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        for idx in range(len(self.X_train)):
            img = self.X_train[idx]
            img = np.stack((img, img, img), axis=-1)
            img = img / 255.0
            img = np.transpose(img, (2, 0, 1)).copy()
            img = torch.from_numpy(img).type(torch.FloatTensor)
            keypoints = self.y_train[idx].astype('float32')

        return img, keypoints

# Create a train dataset



In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan, strategy='mean')

for i in range(30):
    imputer.fit(y_train[:,i].reshape(-1,1))
    y_train[:,i]=(imputer.transform(y_train[:,i].reshape(-1,1))).reshape(-1,)


In [ ]:
X_train=X_train.reshape(X_train.shape[0],96,96)
X_test_submit=X_test_submit.reshape(X_test_submit.shape[0],96,96)
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap='gray')
plt.title("Input Image")
plt.savefig('plot.png', bbox_inches='tight')
plt.show()

In [ ]:
train_points = [y_train[k].reshape([15,2]) for k in range(y_train.shape[0]) ]
train_images = [X_train[k].reshape([96,96]) for k in range(X_train.shape[0])]

In [ ]:
def augment1(img, pnts, rot_deg, zoom_factor, x_shift_pix, y_shift_pix):
    sz = img.shape[-2:]
    def get_rotation(x):
        mysz = x.new_ones(x.shape[0])
        rot_rad = torch.ones_like(mysz)*(rot_deg / 180.0 * np.pi)
        m11 = rot_rad.cos() / zoom_factor
        m12 = rot_rad.sin() / zoom_factor
        t0 = torch.ones_like(mysz)*(x_shift_pix/48.0)
        t1 = torch.ones_like(mysz)*(y_shift_pix/48.0)
        return affine_mat(m11, m12, t0, -m12, m11, t1)
    t1 = AffineCoordTfm(aff_fs=get_rotation, size=sz)
    p1 = Pipeline(funcs=t1)
    x = TensorImage(img).view([1,1,96,96])
    y = TensorPoint(pnts, img_size=[96,96]).view([1,15,2])
    x,y = p1((x,y/48.0-1.0))
    y = y.view([15,2])
    coord_ok = (y[:,0] > -1.0) & (y[:,0] < 1.0) & (y[:,1] > -1.0) & (y[:,1] < 1.0)
    coord_ok = torch.stack([coord_ok, coord_ok], dim=1)
    y = y.where(coord_ok, tensor(np.nan))
    y = y*48.0+48.0
    return np.array(x.view([96,96])), np.array(y)

In [ ]:
import cv2
import numpy as np

def augment(img, pnts, rot_deg, zoom_factor, x_shift_pix, y_shift_pix):

    # Rotate the image
    rotation_matrix = cv2.getRotationMatrix2D((48, 48), rot_deg, zoom_factor)
    img = cv2.warpAffine(img, rotation_matrix, dsize=(96, 96))

    # Shift the image
    translation_matrix = np.float32([[1, 0, x_shift_pix], [0, 1, y_shift_pix]])
    img = cv2.warpAffine(img, translation_matrix, dsize=(96, 96))

    # Augment the point landmarks
    pnts = np.dot(pnts, rotation_matrix.T)
    pnts[:, 0] += x_shift_pix
    pnts[:, 1] += y_shift_pix

    return img, pnts


In [ ]:
augs = []
one_pixel = 2.0/96.0
for dx in range(21):
    for dy in range(21):
        if dx==10 and dy==10:
            continue
        augs.append([0.0, 1.0, dx-10.0, dy-10.0])
for rot_deg in range(21):
    if rot_deg==10:
        continue
    augs.append([rot_deg-10, 1.0, 0.0, 0.0])
for scale in range(21):
    if scale==10:
        continue
    augs.append([0.0, 0.9 + 0.01*scale, 0.0, 0.0])
print(len(augs))

In [ ]:
aug_images = []
aug_points = []
aug_ind = 0
for k1 in range(len(train_images)):
    img,pnt = augment(train_images[k1], train_points[k1], *augs[aug_ind])
    aug_images.append(train_images[k1])
    aug_points.append(train_points[k1])
    aug_images.append(img)
    aug_points.append(pnt)
    aug_ind = (aug_ind + 1) % len(augs)
print(len(aug_images))

In [ ]:
from sklearn.model_selection import train_test_split
from fastai.vision.all import ImageDataLoaders,DataLoader
import tensorflow as tf
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
resize_transform = transforms.RandomResizedCrop((96, 96))
# Load your training data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
train_dataset=TrainDataset(X_train,y_train)
test_dataset=TrainDataset(X_test,y_test)
#train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
#test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)
# Create an ImageDataBunch object from the training and test data
#dataloaders = ImageDataLoaders(X_train, X_test, y_train, y_test,bs=64)
train_dls = DataLoader(train_dataset, batch_size=64,shuffle=True)
test_dls = DataLoader(test_dataset, batch_size=64,shuffle=True)

In [ ]:
import torchvision
import torch
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Linear(model.fc.in_features, 30)

model = model.type(torch.FloatTensor)


In [ ]:
from torch import nn, optim

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01, weight_decay=0.001)
model.train()
# Iterate over the training data
for epoch in range(25):
    for i,(x,y) in enumerate(train_dls):
        # Forward pass the data through the model

        outputs = model(x)

        # Calculate the loss
        loss = criterion(outputs,y)

        # Backpropagate the loss through the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss
        if i % 2== 0:
            print('Epoch: {} | Batch: {} | Loss: {}'.format(epoch, i, loss.item()))

In [ ]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
def evaluate_model(model, test_loader):

  model.eval()

  correct = 0
  total = 0
  with torch.no_grad():
    for batch,(images, labels) in test_dls:
      # Forward pass
      outputs = model(images)

      # Calculate the predicted labels
      _, predicted = torch.max(outputs.data, 1)

      # Count the number of correct predictions
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  # Calculate the accuracy
  accuracy = 100 * correct / total

  return accuracy

accuracy = evaluate_model(model, test_loader)

# Print the accuracy
print('Accuracy on the test data: {:.2f}%'.format(accuracy))

In [ ]:
# Evaluate the model on the test data
total_loss = 0
accuracy = 0
total = 0
model.eval()
with torch.no_grad():
    for batch,(imgs,vals) in enumerate(test_dls):
        outputs = model(imgs)
        loss = criterion(outputs, vals)

        total_loss += loss.item()

        # Calculate the accuracy
        # by averaging the absolute difference between
        # the predicted and actual values

        abs_diff = torch.abs(outputs - vals)/vals
        mean_abs_diff = torch.mean(1-abs_diff, dim=1)
        accuracy +=  mean_abs_diff

        total += vals.size(0)

# Calculate the average loss and accuracy
average_loss = total_loss / len(test_dls)
accuracy = 100 * accuracy

# Print the results
print('Test loss: {}'.format(average_loss))
print('Test accuracy: {}%'.format(accuracy))

In [ ]:

cols = pd.read_csv('/kaggle/input/keypoint-identification-cv-contingent-selections/sample_submission.csv').iloc[0,0:30].index
preds = pd.DataFrame(columns=cols)
# Iterate over the test data and make predictions
for i in range(len(X_test_submit)):
    image = X_test_submit[i]
    image = image.reshape([96, 96])
    image = np.stack((image, image, image), axis=-1)
    image = image / 255.0
    image = np.transpose(image, (2, 0, 1)).copy()

    image = torch.from_numpy(image).type(torch.FloatTensor)
    image = image.unsqueeze(0)
    with torch.no_grad():
        out = model(image).cpu().detach().numpy().reshape((30,))

    # Add the predictions to the DataFrame
    preds.loc[i]=out
ID=[]
for i in range(1,1501):
    ID.append(i)
preds['ID']=ID
print (preds.shape)
preds.to_csv('submission.csv',index = False)